In [1]:
import ipywidgets as widgets

`Output`小部件可以捕获并显示由IPython生成的stdout，stderr和丰富的输出。也可以将输出直接附加到输出窗口小部件，或以编程方式清除输出。

In [2]:
out = widgets.Output(layout={'border': '1px solid black'})
out

Output(layout=Layout(border='1px solid black'))

小部件创建完成后，使用上下文管理器直接输出到它。 您可以将文本打印到输出区域：

In [3]:
with out:
    for i in range(10):
        print(i, 'Hello world!')

In [4]:
import time
time.sleep(60)
out.clear_output()

丰富的输出也可以指向输出区域。 在Jupyter笔记本中很好地显示的任何内容也都可以很好地显示在输出`Output`小部件中。

In [5]:
from IPython.display import YouTubeVideo
with out:
    display(YouTubeVideo('eWzY2nGfkXk'))

我们甚至可以在输出小部件中显示复杂的mimetypes，例如嵌套小部件。

In [6]:
with out:
    display(widgets.IntSlider())

我们还可以直接使用便捷方法`append_stdout`，`append_stderr`或`append_display_data`将输出附加到输出窗口小部件。

In [7]:
out = widgets.Output(layout={'border': '1px solid black'})
out.append_stdout('Output appended with append_stdout')
out.append_display_data(YouTubeVideo('eWzY2nGfkXk'))
out

Output(layout=Layout(border='1px solid black'), outputs=({'output_type': 'stream', 'name': 'stdout', 'text': '…

请注意，`append_display_data`目前无法用于显示小部件。可以在这个[主题](https://github.com/jupyter-widgets/ipywidgets/issues/1811)中跟踪这个错误的状态。

我们可以通过在上下文管理器中使用`IPython.display.clear_output`来清除输出，或者我们可以直接调用小部件的`clear_output`方法。

In [8]:
out.clear_output()

`clear_output`支持关键字参数`wait`。将此设置为`True`，小部件内容不会立即清除。相反，它们在小部件下次收到要显示的内容时被清除。在替换输出小部件中的内容时，这可能很有用：它可以避免在调用`clear_output`之后调整小部件的大小，从而实现更平滑的转换。

最后，我们可以使用输出小部件来捕获使用`capture`装饰器的函数产生的所有输出。

In [9]:
@out.capture()
def function_with_captured_output():
    print('This goes into the output widget')
    raise Exception('As does this')

function_with_captured_output()

`out.capture`支持关键字参数`clear_output`。如果将此设置为`True`，则每次调用该函数时都会清除输出窗口小部件，以便您只能看到最后一次调用的输出。将`clear_output`设置为True时，也可以传递`wait = True`参数，以便只有在新输出可用时清除输出。当然，您也可以随时手动清除输出。

In [10]:
out.clear_output()

## 输出小部件作为交互的基础

输出小部件构成了如何实现交互和相关方法的基础。它也可以用于创建带有小部件和代码输出的丰富布局。定制交互式用户界面看起来如何的一个简单方法是使用`interactive_output`函数将控件挂接到其输出在返回的输出小部件中捕获的函数。在下一个示例中，我们垂直堆叠控件，然后将函数的输出放在右侧。

In [11]:
a = widgets.IntSlider(description='a')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])

## 使用输出小部件调试回调中的错误

在某些平台上，如JupyterLab，由窗口小部件回调生成的输出（例如，附加到小部件特征上的`.observe`方法的函数或按钮小部件上的`.on_click`方法）不会显示在任何位置。即使在其他平台上，也不清楚此输出应该出现在什么单元中。这可能会使回调函数中的调试错误更具挑战性。

访问小部件回调输出的有效工具是使用输出小部件的捕获方法来装饰回调。然后，您可以在新单元中显示该小部件以查看回调输出。

In [12]:
debug_view = widgets.Output(layout={'border': '1px solid black'})

@debug_view.capture(clear_output=True)
def bad_callback(event):
    print('This is about to explode')
    return 1.0 / 0.0

button = widgets.Button(
    description='click me to raise an exception',
    layout={'width': '300px'}
)
button.on_click(bad_callback)
button

Button(description='click me to raise an exception', layout=Layout(width='300px'), style=ButtonStyle())

In [13]:
debug_view

Output(layout=Layout(border='1px solid black'), outputs=({'output_type': 'stream', 'text': 'This is about to e…

## 将输出小部件与日志记录模块集成

While using the .capture decorator works well for understanding and debugging single callbacks, it does not scale to larger applications. Typically, in larger applications, one might use the logging module to print information on the status of the program. However, in the case of widget applications, it is unclear where the logging output should go.

A useful pattern is to create a custom handler that redirects logs to an output widget. The output widget can then be displayed in a new cell to monitor the application while it runs.

虽然使用`.capture`修饰器可以很好地理解和调试单个回调，但它不会扩展到更大的应用程序。通常，在较大的应用程序中，可以使用日志记录模块打印有关程序状态的信息。但是，在小部件应用程序的情况下，不清楚日志记录输出的位置。

一个有用的模式是创建一个将日志重定向到输出小部件的自定义处理程序。然后可以将输出小部件显示在新单元中，以在运行时监视应用程序。

In [14]:
import ipywidgets as widgets
import logging

class OutputWidgetHandler(logging.Handler):
    """ Custom logging handler sending logs to an output widget """

    def __init__(self, *args, **kwargs):
        super(OutputWidgetHandler, self).__init__(*args, **kwargs)
        layout = {
            'width': '100%',
            'height': '160px',
            'border': '1px solid black'
        }
        self.out = widgets.Output(layout=layout)

    def emit(self, record):
        """ Overload of logging.Handler method """
        formatted_record = self.format(record)
        new_output = {
            'name': 'stdout',
            'output_type': 'stream',
            'text': formatted_record+'\n'
        }
        self.out.outputs = (new_output, ) + self.out.outputs

    def show_logs(self):
        """ Show the logs """
        display(self.out)

    def clear_logs(self):
        """ Clear the current logs """
        self.out.clear_output()


logger = logging.getLogger(__name__)
handler = OutputWidgetHandler()
handler.setFormatter(logging.Formatter('%(asctime)s  - [%(levelname)s] %(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [15]:
handler.show_logs()

Output(layout=Layout(border='1px solid black', height='160px', width='100%'))

In [16]:
handler.clear_logs()
logger.info('Starting program')

try:
    logger.info('About to try something dangerous...')
    1.0/0.0
except Exception as e:
    logger.exception('An error occurred!')

## 与来自后台线程的输出小部件交互

当显示后台线程产生的输出时，Jupyter的`display`机制可能不直观。后台线程的输出被打印到主线程正在写入的任何单元格中。要直接看到这个，创建一个重复打印到标准输出的线程：

In [19]:
import threading
import time
import itertools

def run():
    for i in itertools.count(0):
        time.sleep(1)
        print(('output from background {}'.format(i)))


t = threading.Thread(target=run)
t.start()

output from background 0
output from background 1
output from background 2
output from background 3
output from background 4
output from background 5
output from background 6
output from background 7
output from background 8
output from background 9
output from background 10
output from background 11
output from background 12
output from background 13
output from background 14
output from background 15
output from background 16
output from background 17
output from background 18
output from background 19
output from background 20
output from background 21
output from background 22
output from background 23
output from background 24
output from background 25
output from background 26
output from background 27
output from background 28
output from background 29
output from background 30
output from background 31
output from background 32
output from background 33
output from background 34
output from background 35
output from background 36
output from background 37
output from background

这总是打印在当前活动的单元格中，而不是启动后台线程的单元格。

这可能会导致输出小部件中令人惊讶的行为。 在输出窗口小部件捕获输出的期间，无论线程如何，笔记本中生成的任何输出都将进入输出窗口小部件。

避免意外的最好方法是不要在多线程生成输出的上下文中使用输出窗口小部件的上下文管理器。 相反，我们可以将输出小部件传递给在线程中执行的函数，并使用`append_display_data()`，`append_stdout()`或`append_stderr()`方法将可显示的输出附加到输出小部件。

In [20]:
import threading
from IPython.display import display, HTML
import ipywidgets as widgets
import time

def thread_func(something, out):
    for i in range(1, 5):
        time.sleep(0.3)
        out.append_stdout('{} {} {}\n'.format(i, '**'*i, something))
    out.append_display_data(HTML("<em>All done!</em>"))

display('Display in main thread')
out = widgets.Output()
# Now the key: the container is displayed (while empty) in the main thread
display(out)

thread = threading.Thread(
    target=thread_func,
    args=("some text", out))
thread.start()

'Display in main thread'

Output()

output from background 172
output from background 173
output from background 174
output from background 175
output from background 176
output from background 177
output from background 178
output from background 179
output from background 180
output from background 181
output from background 182
output from background 183
output from background 184
output from background 185
output from background 186
output from background 187
output from background 188
output from background 189
output from background 190
output from background 191
output from background 192
output from background 193
output from background 194
output from background 195
output from background 196
output from background 197
output from background 198
output from background 199
output from background 200
output from background 201
output from background 202
output from background 203
output from background 204
output from background 205
output from background 206
output from background 207
output from background 208
o